In [157]:
from vertexai.preview.generative_models import GenerativeModel
from google import genai
from google.genai import types

In [158]:
from vertexai import init

In [159]:
init(project="qwiklabs-gcp-02-8fc93094313c", location="us-central1")

In [160]:
model = "gemini-2.5-pro-preview-06-05"

In [161]:
gemini_client = genai.Client(
    vertexai=True,
    project="qwiklabs-gcp-02-8fc93094313c",
    location="global",
)

# Classification based on Category

In [162]:
classification_prompt = (
        "Classify the following question into one of these categories: "
        "Employment, General Information, Emergency Services, or Tax Related.\n\n"
        f"for Questions respond with above category \"\n\n"
        "Respond only with the category name."
    )

In [163]:
def classify_question(question: str):

    response = gemini_client.models.generate_content(
    model = model,
    contents = [f"Input Prompt: ${question}"],
    config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 1,
        system_instruction=[types.Part.from_text(text=classification_prompt)]
      )
    )
    return response.text


In [164]:
#test
print(classify_question('who can apply for govt job'))

Employment


In [165]:
import pytest

Added testcases to validate the response came back from the model is same or not

In [166]:
def test_employment():
    result = classify_question("How do I apply for a job?")
    assert result == "Employment"

def test_general_information():
    result = classify_question("What are your office hours?")
    assert result == "General Information"

def test_emergency_services():
    result = classify_question("There's a accident! Call 108!")
    assert result == "Emergency Services"

def test_tax_related():
    result = classify_question("How do I file my taxes?")
    assert result == "Tax Related"

In [167]:
test_employment()
test_general_information()
test_emergency_services()
test_tax_related()

# Government announcement posts

In [186]:
system_prompt = """Write a short government social media post (under 100 characters) with clear, factual language and include the hashtag #breakingnews."""

def generate_announcement_post(text):

    response = gemini_client.models.generate_content(
    model = model,
    contents = [f"Input Prompt: ${text}"],
    config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 1,
        system_instruction=[types.Part.from_text(text=system_prompt)]
      )
    )
    return response.text
    return response.text.strip()

In [187]:
post = generate_announcement_post("Due to heavy raining, all schools and offices will be closed for next 2 days")

In [188]:
print(post)

All schools & offices closed for 2 days due to heavy rain. #breakingnews


In [189]:
def does_tweet_follow_rules(tweet):
  response = gemini_client.models.generate_content(
    model = model,
    contents = [f"Input Prompt: ${tweet}"],
    config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 1,
        system_instruction=[types.Part.from_text(text= """Does the tweet follow the following rules:
          1. Keep your Tweets below 100 characters
          2. Include the hashtag #breakingnews at the end of every tweet
          Only return Yes or No
          Tweet: {0}
          Output: """.format(tweet))]
      )
    )
  return response.text.strip()

In [190]:
print(does_tweet_follow_rules('Schools & offices will be closed for the next 2 days due to heavy rain. Stay safe. #breakingnews'))

Yes


In [191]:
def test_tweet_results_1():
  generated_tweet = generate_announcement_post("Due to independence day all public and private schools will be given holiday")
  correct = does_tweet_follow_rules(generated_tweet)
  assert correct == "Yes"
def test_does_not_follow_rules():
  generated_tweet = "Schools are now on summer vacation for the next two months."
  correct = does_tweet_follow_rules(generated_tweet)
  assert correct == "Yes"

In [192]:
test_tweet_results_1()
test_does_not_follow_rules()

AssertionError: 

In [180]:
from vertexai.evaluation import (
    EvalTask,
    PointwiseMetric,
    PairwiseMetric,
    PointwiseMetricPromptTemplate,
    PairwiseMetricPromptTemplate,
    MetricPromptTemplateExamples,
)
from vertexai.generative_models import (
    GenerativeModel,
    HarmCategory,
    HarmBlockThreshold,
)
import pandas as pd
import plotly.graph_objects as go
from IPython.display import HTML, Markdown, display
import datetime

In [181]:
posts = [
    {
        "prompt": "Extreme heat warning issued for Lucknow from June 20 to June 25. Citizens are advised to stay indoors during peak hours and drink plenty of water.",
        "post": "Heat alert in Lucknow, June 20–25! Stay indoors 12–4 PM and keep hydrated. #Heatwave #Lucknow"
    },
    {
        "prompt": "Government offices and banks will be closed on October 2 in observance of Gandhi Jayanti.",
        "post": "Offices and banks closed on Oct 2 for Gandhi Jayanti. Plan your errands ahead. #GandhiJayanti #PublicHoliday"
    },
    {
        "prompt": "All schools in Ahmedabad will reopen on June 10 after the summer break. Students must carry water bottles and wear light clothing.",
        "post": "Ahmedabad schools reopen June 10! Stay cool, stay safe. #SchoolReopening #Ahmedabad"
    },
    {
        "prompt": "A rise in dengue cases has been observed in Kochi. Avoid mosquito breeding and use repellents regularly.",
        "post": "Dengue cases rising in Kochi. Drain stagnant water and use repellent. #DengueAlert #KochiHealth"
    },
    {
        "prompt": "Metro services on the Red Line will be suspended on July 15 from 10 AM to 4 PM due to track repairs.",
        "post": "Red Line Metro closed July 15, 10 AM–4 PM. Use alternate routes. #MetroUpdate #CityTransit"
    },
    {
        "prompt": "Scheduled water supply interruption in East Delhi on July 1 from 6 AM to 3 PM due to pipeline work.",
        "post": "No water in East Delhi on July 1, 6 AM–3 PM. Please store water in advance. #WaterAlert #DelhiNotice"
    },
    {
        "prompt": "Garbage collection in Bhopal will be delayed by a day this week due to the festival holiday on Tuesday.",
        "post": "Bhopal garbage pickup delayed this week—Tuesday's holiday shifts collection by a day. #BhopalUpdate #CivicAlert"
    },
    {
        "prompt": "Indore will host a tree plantation drive on August 15. Volunteers can register on the city portal.",
        "post": "Tree plantation drive in Indore on Aug 15! Join the green mission. Register on the city site. #GreenIndore #PlantATree"
    }
]

## Creating Data set

In [182]:
data = [
    {
        "instruction": system_prompt,
        "context": f"announcement: {item['prompt']}",
        "response": item["post"],
    } for item in posts
]

eval_dataset = pd.DataFrame(data)

In [183]:
run_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
        MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
        MetricPromptTemplateExamples.Pointwise.VERBOSITY,
        MetricPromptTemplateExamples.Pointwise.INSTRUCTION_FOLLOWING,
        MetricPromptTemplateExamples.Pointwise.SAFETY
    ],
    experiment=f"govt-announcement-posts-{run_ts}"
)


## Running Evaluation

In [184]:
prompt_template = (
    "Instruction: {instruction}. Prompt: {context}. Post: {response}"
)
result = eval_task.evaluate(
      prompt_template=prompt_template,
      experiment_run_name=f"generate-announcemet-post-{run_ts}"
)
evaluation_results = []
evaluation_results.append(result)

INFO:vertexai.evaluation.eval_task:Logging Eval Experiment metadata: {'prompt_template': 'Instruction: {instruction}. Prompt: {context}. Post: {response}'}
INFO:vertexai.evaluation._evaluation:Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 32 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 32/32 [00:03<00:00,  8.20it/s]
INFO:vertexai.evaluation._evaluation:All 32 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:3.917346769998403 seconds


## Results

In [185]:
from vertexai.preview.evaluation import notebook_utils
notebook_utils.display_eval_result(eval_result=result)

### Summary Metrics

,row_count,groundedness/mean,groundedness/std,verbosity/mean,verbosity/std,instruction_following/mean,instruction_following/std,safety/mean,safety/std
0,8.0,1.0,0.0,-0.125,0.353553,3.0,0.0,1.0,0.0


### Row-based Metrics

,instruction,context,response,prompt,groundedness/explanation,groundedness/score,verbosity/explanation,verbosity/score,instruction_following/explanation,instruction_following/score,safety/explanation,safety/score
0,You are a professional communications reviewer...,announcement: Extreme heat warning issued for ...,"Heat alert in Lucknow, June 20–25! Stay indoor...",Instruction: You are a professional communicat...,The response is completely grounded in the pro...,1.0,"The response is perfectly concise, providing a...",0.0,"The post is somewhat clear and concise, but it...",3.0,The response is safe as it does not contain an...,1.0
1,You are a professional communications reviewer...,announcement: Government offices and banks wil...,Offices and banks closed on Oct 2 for Gandhi J...,Instruction: You are a professional communicat...,"The response is fully grounded, as it only con...",1.0,"The response is perfectly concise, providing a...",0.0,"The post is somewhat clear and concise, but it...",3.0,The response is safe because it does not conta...,1.0
2,You are a professional communications reviewer...,announcement: All schools in Ahmedabad will re...,"Ahmedabad schools reopen June 10! Stay cool, s...",Instruction: You are a professional communicat...,The response is fully grounded in the provided...,1.0,"The response is appropriately concise, providi...",0.0,The response partially fulfills the instructio...,3.0,The response is safe as it does not contain an...,1.0
3,You are a professional communications reviewer...,announcement: A rise in dengue cases has been ...,Dengue cases rising in Kochi. Drain stagnant w...,Instruction: You are a professional communicat...,The generated response is fully grounded in th...,1.0,"The response is appropriately concise, providi...",0.0,The response is a good start but missed to add...,3.0,The response is safe as it does not contain an...,1.0
4,You are a professional communications reviewer...,announcement: Metro services on the Red Line w...,"Red Line Metro closed July 15, 10 AM–4 PM. Use...",Instruction: You are a professional communicat...,The response is fully grounded because it only...,1.0,"The response is perfectly concise, providing a...",0.0,"The post is concise and understandable, but mi...",3.0,The response is safe because it does not conta...,1.0
5,You are a professional communications reviewer...,announcement: Scheduled water supply interrupt...,"No water in East Delhi on July 1, 6 AM–3 PM. P...",Instruction: You are a professional communicat...,The response is fully grounded since all the i...,1.0,"The response is appropriately concise, providi...",0.0,"The post is clear, concise, and provides neces...",3.0,The response is safe as it does not contain ha...,1.0
6,You are a professional communications reviewer...,announcement: Garbage collection in Bhopal wil...,Bhopal garbage pickup delayed this week—Tuesda...,Instruction: You are a professional communicat...,The response only uses information provided in...,1.0,"The response is perfectly concise, providing a...",0.0,"The post is mostly clear and concise, but it m...",3.0,The response is safe as it does not contain an...,1.0
7,You are a professional communications reviewer...,announcement: Indore will host a tree plantati...,Tree plantation drive in Indore on Aug 15! Joi...,Instruction: You are a professional communicat...,"The response is fully grounded, as it only con...",1.0,"The response is slightly brief, missing the re...",-1.0,"The post is concise and clear, but misses cruc...",3.0,"The response does not contain any hate speech,...",1.0
